In [ ]:
import cv2
import time
from ultralytics import YOLO

# Define your emotion class names
emotion_classes = ['angry','boring','disgust','fear','happy','neutral','sad','stress','surprise']

# Load your custom model
model = YOLO("D:\RuhunaNew\Academic\Research\Facial_Recog\Group_50\\best.pt")

cap = cv2.VideoCapture(0)
inference_interval = 5  # seconds
last_inference_time = 0
last_label = "No emotion detected"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    # Run inference every 5 seconds
    if current_time - last_inference_time >= inference_interval:
        results = model(frame)
        last_inference_time = current_time

        # Process results
        if results[0].boxes is not None and len(results[0].boxes) > 0:
            box = results[0].boxes[0]
            class_id = int(box.cls[0])
            confidence = box.conf[0].item()
            last_label = f"{emotion_classes[class_id]} ({confidence:.2f})"
            # Optionally, draw box:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, last_label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        else:
            last_label = "No emotion detected"

    # Overlay the last detected label on the frame
    cv2.putText(frame, last_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("YOLOv11 Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()